In [131]:
import keras
from keras.models import load_model
import numpy as np
import random

In [132]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [133]:
maxlen = 40
text = open('hamlet.txt').read().lower()  # read the file and convert to lowercase
chars = sorted(list(set(text)))
# what position does each character exist at in the prev list
char_indices = dict((c,i) for i, c in enumerate(chars))
indices_char = dict((i,c) for i,c in enumerate(chars))

In [134]:
model = load_model('model.h5')

In [135]:
# random.seed(1)
start_index = 0 # random.randint(0, len(text) - maxlen - 1)
diversity = 0.5
generated = ''
sentence = text[start_index: start_index + maxlen]

generated += sentence
for i in range(50):
    x = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x[0, t, char_indices[char]] = 1.

    preds = model.predict(x, verbose=0)[0]
    next_index = sample(preds, diversity)
    next_char = indices_char[next_index]
    generated += next_char
    sentence = sentence[1:] + next_char
print(generated)

hamlet, prince of denmark

by william shall his marry and her bedomen preases,
and the mos
